---
title: "Output to Excel with `xlwings`"
bibliography: "../../blog.bib"
author: "Peter Amerkhanian"
description: "Computationally producing Excel output with `xlwings`"
date: "2025-3-12"
image: thumbnail.png
draft: true
categories: ['Python', 'Data Management']
format:
  html:
    toc: true
    toc-depth: 3
    code-fold: false
    code-tools: true
---

In [157]:
import pandas as pd
import numpy as np
import xlwings as xw
import os

In [158]:
#| code-fold: true
def simulate_df(num_transactions=1000):
    np.random.seed(1)
    customer_ids = np.random.randint(1000, 5000, num_transactions)
    transaction_amounts = np.round(np.random.uniform(5, 500, num_transactions), 2)
    payment_methods = np.random.choice(["Credit Card", "Debit Card", "PayPal", "Cash"], num_transactions)
    categories = np.random.choice(["Electronics", "Clothing", "Groceries", "Entertainment"], num_transactions)
    transaction_dates = pd.date_range(start="2024-01-01", periods=num_transactions, freq="D")
    data = {
        "transaction_id": range(1, num_transactions + 1),
        "customer_id": customer_ids,
        "amount": transaction_amounts,
        "payment_method": payment_methods,
        "category": categories,
        "date": transaction_dates
    }

    df = pd.DataFrame(data)
    return df

In [159]:
df = simulate_df()
df.head()

,transaction_id,customer_id,amount,payment_method,category,date
0,1,2061,177.75,Debit Card,Electronics,2024-01-01
1,2,1235,33.20,Credit Card,Clothing,2024-01-02
2,3,4980,118.26,Credit Card,Groceries,2024-01-03
3,4,2096,333.73,Debit Card,Entertainment,2024-01-04
4,5,4839,251.14,Credit Card,Electronics,2024-01-05


In [160]:
dollars_by_method = df.pivot_table(
    index="payment_method",
    columns="category",
    values="amount",
    aggfunc="sum"
)
dollars_by_method.index.name = "Payment Method"
dollars_by_method.columns.name = 'Category'
dollars_by_method

Category,Clothing,Electronics,Entertainment,Groceries
Payment Method,,,,
Cash,15016.00,17026.47,18208.16,18242.88
Credit Card,16074.92,10960.64,16052.85,20187.25
Debit Card,14032.23,17325.96,14363.99,17188.67
PayPal,14143.84,15108.84,16852.17,12065.14


In [161]:
dollars_by_method.loc[:, 'Total'] = dollars_by_method.sum(axis=1)
dollars_by_method.loc['Total', :] = dollars_by_method.sum(axis=0)
dollars_by_method

Category,Clothing,Electronics,Entertainment,Groceries,Total
Payment Method,,,,,
Cash,15016.00,17026.47,18208.16,18242.88,68493.51
Credit Card,16074.92,10960.64,16052.85,20187.25,63275.66
Debit Card,14032.23,17325.96,14363.99,17188.67,62910.85
PayPal,14143.84,15108.84,16852.17,12065.14,58169.99
Total,59266.99,60421.91,65477.17,67683.94,252850.01


In [162]:
filename = "test.xlsx"
if os.path.exists(filename):
        wb = xw.Book(filename)
else:
    wb = xw.Book()
    wb.save(filename)
wb

<Book [test.xlsx]>

In [163]:
sheet_name = 'first_test'
try:
    sheet_new = wb.sheets.add(sheet_name) 
except ValueError:
    sheet_new = wb.sheets[sheet_name]
sheet_new

<Sheet [test.xlsx]first_test>

In [ ]:
sheet_new.range("A1").options(index=True, header=True).value = dollars_by_method

In [165]:
wb.save()

![](1.png)

In [ ]:
sheet_new.used_range.api.EntireColumn.AutoFit()
sheet_new.used_range.api.EntireRow.AutoFit()
wb.save()

![](2.png)

In [167]:
headers = sheet_new.range("A1").expand("right").value
headers

['Payment Method',
 'Clothing',
 'Electronics',
 'Entertainment',
 'Groceries',
 'Total']

In [168]:
col_index = headers.index('Clothing') + 1
col_letter = xw.utils.col_name(col_index) 
col_letter

'B'

In [169]:
(sheet_new
 .range(f"{col_letter}2:{col_letter}1048576")
 .number_format) = "$#,##0.00"
wb.save()

![](3.png)

In [170]:
for col in dollars_by_method.columns:
    col_index = headers.index(col) + 1
    col_letter = xw.utils.col_name(col_index)
    (sheet_new
     .range(f"{col_letter}2:{col_letter}1048576")
     .number_format) = "$#,##0.00"
wb.save()

![](4.png)

In [171]:
import string
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [172]:
end_col = string.ascii_uppercase[dollars_by_method.reset_index().shape[1]-1]
header_range = sheet_new.range(f"A1:{end_col}1")
header_range.font.bold = True
wb.save()

![](5.png)

In [173]:
end_row = dollars_by_method.reset_index().shape[0] + 1
header_range = sheet_new.range(f"A1:A{end_row}")
header_range.font.bold = True
wb.save()

![](6.png)

In [174]:
sheet_new.range(f"{end_col}1:{end_col}{end_row}").color = (242, 242, 242)
sheet_new.range(f"A{end_row}:{end_col}{end_row}").color = (242, 242, 242)
wb.save()

![](7.png)

In [175]:
used_range = sheet_new.used_range
used_range

<Range [test.xlsx]first_test!$A$1:$F$6>

In [176]:
used_range.api.Borders.LineStyle = 1
used_range.api.Borders.Weight = 2
wb.save()

https://stackoverflow.com/a/73015199

![](8.png)

In [177]:
dollars_by_method

Category,Clothing,Electronics,Entertainment,Groceries,Total
Payment Method,,,,,
Cash,15016.00,17026.47,18208.16,18242.88,68493.51
Credit Card,16074.92,10960.64,16052.85,20187.25,63275.66
Debit Card,14032.23,17325.96,14363.99,17188.67,62910.85
PayPal,14143.84,15108.84,16852.17,12065.14,58169.99
Total,59266.99,60421.91,65477.17,67683.94,252850.01


In [194]:
#| code-fold: true

def get_or_create_workbook(filename: str,
                           display_alerts: bool = False,
                           screen_updating: bool= False):
    """
    Checks if the specified Excel file exists. If it does, opens it; 
    otherwise, creates a new one. Returns the workbook object.

    Args:
        filename (str): The name of the Excel file. Default is "output.xlsx".

    Returns:
        xlwings.Book: The opened or newly created workbook.
    """
    if os.path.exists(filename):
        wb = xw.Book(filename)
        wb.app.display_alerts = display_alerts
        wb.app.screen_updating = screen_updating
    else:
        wb = xw.Book()
        wb.app.display_alerts = display_alerts
        wb.app.screen_updating = screen_updating
        wb.save(filename)
    return wb

def select_sheet(name: str, wb: xw.Book):
    """
    Selects an existing sheet by name or creates a new one if it does not exist.

    Args:
        name (str): The name of the sheet to select or create.
        wb (xlwings.Book): The Excel workbook object.

    Returns:
        xlwings.Sheet: The selected or newly created sheet."
    """
    try:
        sheet_new = wb.sheets.add(name) 
    except ValueError:
        sheet_new = wb.sheets[name]
    return sheet_new

def write_df_to_excel(df: pd.DataFrame,
                      sheet: xw.Sheet,
                      cell_start: str="A1",
                      bold_indexes: bool=True):
    """
    Writes a pandas DataFrame to an Excel sheet and bolds the index and column headers.
    
    Args:
        df (pd.DataFrame): The DataFrame to write.
        sheet (xlwings.Sheet): The Excel sheet where the DataFrame will be written.
    """
    # Write the DataFrame to the Excel sheet starting from cell A1
    sheet.range(cell_start).options(index=True, header=True).value = df
    
    if bold_indexes:
        # Get the range for the index
        index_start_row = 1  # Start from row 2 (since the header is in row 1)
        index_end_row = index_start_row + df.shape[0]
        # Get the range for the headers
        header_start_col = 1  # Start from column B (the first column is the index)
        header_end_col = header_start_col + df.reset_index().shape[1] - 1  # Adjust for headers

        # For multi-index columns, we need to get the full range
        if isinstance(df.index, pd.MultiIndex):
            for i in range(df.index.nlevels):
                # Define the range for each level of the multi-index
                level_range = sheet.range(f"{string.ascii_uppercase[i]}{index_start_row}:{string.ascii_uppercase[i]}{index_end_row}")
                level_range.font.bold = True
                index_end_row = index_start_row + df.shape[0]
        else:
            index_range = sheet.range(f"A{index_start_row}:A{index_end_row}")
            index_range.font.bold = True
        if isinstance(df.columns, pd.MultiIndex):
            for i in range(df.columns.nlevels):
                # Define the range for each level of the multi-index
                level_range = sheet.range(f"B{i+1}:{string.ascii_uppercase[header_end_col]}{i+1}")
                level_range.font.bold = True
        else:
            # If it's a single index, bold the header range directly
            header_range = sheet.range(f"B1:{string.ascii_uppercase[header_end_col - 1]}1")
            header_range.font.bold = True


def autofit_all_sheets(wb: xw.Book):
    """
    Autofits all columns in all sheets of the given workbook.
    
    Args:
        wb (xlwings.Book): The Excel workbook object.
    """
    for sheet in wb.sheets:
        if sheet.used_range.columns.count > 1:  # Ensure there's data in the sheet
            sheet.used_range.api.EntireColumn.AutoFit()  # Autofit columns
            sheet.used_range.api.EntireRow.AutoFit()     # Autofit rows

def format_dollar_column(sheet, header_name, format="$#,##0.00", header_row="A"):
    """
    Formats the column with the given header name as a dollar amount in an Excel sheet.

    Args:
        sheet (xlwings.Sheet): The Excel sheet object.
        header_name (str): The column header to search for.
        format (str, optional): The Excel number format for currency. Default is "$#,##0.00".
        header_row (str, optional): The row letter where headers are located. Default is "A".

    Returns:
        None
    """
    # Find the column index based on the header
    headers = sheet.range(f"{header_row}1").expand("right").value  # Read all headers in row 1
    if header_name not in headers:
        raise ValueError(f"Header '{header_name}' not found in the sheet.")
    
    col_index = headers.index(header_name) + 1  # Convert to Excel 1-based index
    col_letter = xw.utils.col_name(col_index)  # Convert to letter (e.g., B, C)
    
    # Apply dollar format (e.g., $1,234.56)
    sheet.range(f"{col_letter}2:{col_letter}1048576").number_format = format

def make_borders(sheet, linestyle=1, weight=2):
    used_range = sheet.used_range
    used_range.api.Borders.LineStyle = linestyle
    used_range.api.Borders.Weight = weight

def close_out_book(wb: xw.Book, autofit: bool=True):
    if autofit:
        autofit_all_sheets(wb)
    if "Sheet1" in [sheet.name for sheet in wb.sheets]:
        wb.sheets["Sheet1"].delete()
    wb.save()
    wb.close()

In [ ]:
wb = get_or_create_workbook("test.xlsx")
function_sheet = select_sheet('function_sheet', wb)
write_df_to_excel(dollars_by_method, function_sheet, bold_indexes=True)
for col in dollars_by_method.columns:
    format_dollar_column(function_sheet, col)
make_borders(function_sheet)
end_col = string.ascii_uppercase[dollars_by_method.reset_index().shape[1]-1]
end_row = dollars_by_method.reset_index().shape[0] + 1
function_sheet.range(f"{end_col}1:{end_col}{end_row}").color = (242, 242, 242)
function_sheet.range(f"A{end_row}:{end_col}{end_row}").color = (242, 242, 242)
close_out_book(wb)

<Range [test.xlsx]function_sheet!$A$1>
<Range [test.xlsx]function_sheet!$A$1:$F$6>
<Range [test.xlsx]function_sheet!$A$1:$F$6>


In [180]:
function_sheet.used_range

<Range [test.xlsx]function_sheet!$A$1:$G$7>

In [181]:
function_sheet.used_range

<Range [test.xlsx]function_sheet!$A$1:$G$7>

In [182]:
df['Year'] = df['date'].dt.year.astype(str)

In [183]:
dollars_by_method_yr = df.pivot_table(
    index=["Year", "payment_method"], 
    columns="category",
    values="amount",
    aggfunc="sum"
)
dollars_by_method_yr.index.names = ["Year", "Payment Method"]
dollars_by_method_yr.columns.name = 'Category'
dollars_by_method_yr = dollars_by_method_yr.fillna(0)
dollars_by_method_yr

Category             Clothing  Electronics  Entertainment  Groceries
Year Payment Method                                                 
2024 Cash             5709.71      5182.41        8661.28    6552.53
     Credit Card      5846.53      2976.99        5334.29    7983.28
     Debit Card       4887.89      6997.97        4998.41    5603.41
     PayPal           5225.22      5649.27        7034.34    3945.90
2025 Cash             4651.67      6148.02        5110.80    7717.81
     Credit Card      6843.13      4618.44        5946.31    5828.80
     Debit Card       5801.20      6606.99        7087.51    4554.53
     PayPal           6203.22      5220.53        5592.94    3442.25
2026 Cash             4654.62      5696.04        4436.08    3972.54
     Credit Card      3385.26      3365.21        4772.25    6375.17
     Debit Card       3343.14      3721.00        2278.07    7030.73
     PayPal           2715.40      4239.04        4224.89    4676.99